In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\Chicken Disease Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\Chicken Disease Classification'

In [60]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class BaseModelConfig:
    root_dir : Path 
    base_model_path : Path
    updated_base_model_path : Path
    params_image_size : list
    params_learning_rate : float
    params_include_top : bool
    params_weights : str
    params_classes : int
    params_batch_size : int
    params_epochs : int

In [61]:
from CDC.constants import *
from CDC.utils.helper_functions import read_yaml, create_directories
from box import ConfigBox

In [64]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> ConfigBox:
        config = self.config.data_ingestion
        create_directories([config.root_dir])    

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            unzip_dir = config.unzip_dir,
            local_data_file = config.local_data_file,
            source_url = config.source_URL
        )

        return data_ingestion_config
    
    def get_base_model_config(self) -> BaseModelConfig:
        config = self.config.base_model
        create_directories([config.root_dir])
        base_model_config = BaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            params_image_size = self.params.INPUT_SHAPE,
            params_batch_size = self.params.BATCH_SIZE,
            params_epochs = self.params.EPOCHS,
            params_learning_rate = self.params.LEARNING_RATE,
            params_classes = self.params.CLASSES,
            params_include_top = self.params.INCLUDE_TOP,
            params_weights = self.params.WEIGHTS
        )
        
        return base_model_config




In [65]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [74]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import CategoricalCrossentropy

class BaseModel:
    def __init__(self, config : BaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = VGG16(
            weights= self.config.params_weights,
            include_top= self.config.params_include_top,
            input_shape=self.config.params_image_size
        )

        self.model.save(self.config.base_model_path)

    
    def _prepare_full_model(self, model,classes,learning_rate,freeze_all = True,freeze_till = None):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif freeze_till:
            for layer in model.layers[:freeze_till]:
                layer.trainable = False

        flatten = Flatten()(model.output)
        predictions = Dense(classes, activation='softmax')(flatten)
        model = Model(inputs=model.input, outputs=predictions)
        model.compile(optimizer=SGD(lr=learning_rate), 
            loss=CategoricalCrossentropy(), 
            metrics=['accuracy'])
        
        return model
        
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            self.model,
            self.config.params_classes,
            self.config.params_learning_rate,
            True,
            None
        )

        self.full_model.save(self.config.updated_base_model_path)
        


    

In [75]:
try:
    configuration_manager = ConfigurationManager()
    print("hello")
    configuration = configuration_manager.get_base_model_config()
    print("got configuration")
    base_model = BaseModel(configuration)
    print("base model instantiated")
    base_model.get_base_model()
    print("got base model")
    base_model.update_base_model()
    print("updated base model")
except Exception as e:
    print(e)


[2023-06-21 02:50:29,443: INFO: helper_functions: yaml file {filename} uploaded successfully]
[2023-06-21 02:50:29,448: INFO: helper_functions: yaml file {filename} uploaded successfully]
[2023-06-21 02:50:29,450: INFO: helper_functions: created directory at: artifacts]
hello
[2023-06-21 02:50:29,453: INFO: helper_functions: created directory at: artifacts/prepare_base_model]
got configuration
base model instantiated
[2023-06-21 02:50:30,303: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
got base model
[2023-06-21 02:50:30,473: WARNING: optimizer: `lr` is deprecated, please use `learning_rate` instead, or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.SGD.]
updated base model
